In [1]:
%pip install --upgrade --quiet  docx2txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%pip install langchain_community
%pip install -qU langchain-text-splitters

In [22]:
%pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install -qU "langchain-chroma>=0.1.2"

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [40]:
%pip install -U langchain langchainhun --quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langchainhun (from versions: none)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for langchainhun


In [2]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=70,
    chunk_overlap=10
)
loader = Docx2txtLoader('./output.docx')
document_list = loader.load_and_split(text_splitter=text_splitter)

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
load_dotenv()

embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')

In [ ]:
%pip install pinecone-client==0.2.1
%pip install langchain-pinecone
%pip install -qU langchain-pinecone pinecone-notebooks

In [59]:
pip show pinecone-client

Name: pinecone-client
Version: 5.0.1
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: c:\Users\suzuk\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: certifi, pinecone-plugin-inference, pinecone-plugin-interface, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [58]:
import os
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'food-index'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key = pinecone_api_key)
index = pc.Index(index_name)
database = PineconeVectorStore.from_documents(document_list,embedding, index=index)

ModuleNotFoundError: No module named 'langchain_pinecone'

In [80]:
from langchain_chroma import Chroma
# database = Chroma.from_documents(documents=document_list,collection_name='chroma-food', persist_directory="./chroma", embedding=embedding)
database = Chroma(collection_name='chroma-food', persist_directory="./chroma", embedding_function=embedding)

In [81]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

c:\Users\suzuk\AppData\Local\Programs\Python\Python313\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [82]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model='gpt-4o')

In [83]:
from langchain.chains import RetrievalQA

retriever=database.as_retriever()

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

In [84]:
retriever.invoke(query)

[Document(metadata={'source': './output.docx'}, page_content='중 가장 시원한 국물을 자랑하는 닭한마리 집. 원래 가려던 집이 문을 닫아서 옆집으로 우연히 들어갔는데 정말 그 시원함을'),
 Document(metadata={'source': './output.docx'}, page_content='먼저 맑은 국물과 담백한 국물이 떠오른다면 추천하고 싶은 곳. 요즘 같은 날씨에 정갈한 곰탕 한그릇으로 몹보신!! 메밀만두도'),
 Document(metadata={'source': './output.docx'}, page_content='리뷰:의정부 민락동에 위치한 곰탕집 먼저 맑은 국물과 담백한 국물이 떠오른다면 추천하고 싶은 곳. 요즘 같은 날씨에 정갈한'),
 Document(metadata={'source': './output.docx'}, page_content='리뷰:여태까지 먹어 보았던 닭 한마리 중 가장 시원한 국물을 자랑하는 닭한마리 집. 원래 가려던 집이 문을 닫아서 옆집으로')]

In [92]:
query = """
식당이름과 위치를 추가적으로 말해주세요
따뜻한 국물이 맛있는 곳을 추천해주세요
"""
ai_message = qa_chain({"query": query})

In [93]:
ai_message

{'query': '\n식당이름과 위치를 추가적으로 말해주세요\n따뜻한 국물이 맛있는 곳을 추천해주세요\n',
 'result': '추천드리는 식당은 의정부 민락동에 위치한 곰탕집입니다. 이곳은 맑고 담백한 국물의 곰탕과 메밀만두가 맛있습니다.'}